In [1]:
from time import perf_counter
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from joblib import Parallel, delayed
import os
import json
import dag_utils as utils
from Baselines import Nonneg_dagma, MetMulDagma
from Baselines import colide_ev
from Baselines import DAGMA_linear
from Baselines import notears_linear
from BUILD import BUILD
from utils import *
PATH = './results/samples/'
SAVE = True 
SEED = 10
N_CPUS = os.cpu_count()  # Get number of available CPUs
np.random.seed(SEED)

In [2]:
# -*- coding: utf-8 -*-
"""
Structured DAG experiments: data simulation -> baselines -> metrics -> save & return.

Assumes the following are available in scope or importable:
  - utils.simulate_sem, utils.standarize, utils.to_bin, utils.count_accuracy, utils.compute_norm_sq_err
  - TopoGreedy_refresh, DAGMA_linear, colide_ev, notears_linear
"""

from __future__ import annotations
import os, json, math, uuid, shutil, datetime as dt
from dataclasses import dataclass, asdict, field
from pathlib import Path
from time import perf_counter
from typing import Callable, Dict, Any, List, Optional, Tuple

import numpy as np
from sklearn.metrics import f1_score
from Baselines import GOLEM_Torch
import matplotlib.pyplot as plt

# ---------------------------------------------
# Utility: lambda schedule
# ---------------------------------------------
def get_lambda_value(n_nodes: int, n_samples: int, times: float = 1.0) -> float:
    """Common λ heuristic: sqrt(log p / n) scaled by `times`."""
    return math.sqrt(max(1e-12, np.log(max(2, n_nodes))) / max(2, n_samples)) * times


# ---------------------------------------------
# Baseline spec & registry
# ---------------------------------------------
@dataclass
class BaselineSpec:
    """
    Unified baseline configuration.

    - model: either a *callable estimator* with .fit() method (class) OR a function (e.g., notears_linear)
    - init: kwargs passed to class constructor (ignored if model is a function)
    - args: kwargs passed to .fit(...) or function call
    - name: label for legends / saving
    - standardize: if True, pass standardized X to the model; else pass raw X
    - adapt_lambda: if True, rescale 'lamb' or 'lambda1' in args per (p, n)
    - topo_transpose: if True, transpose W_est and binarized variant to align with your downstream expectations
                      (e.g., TopoGreedy variants that output transposed)
    - is_topogreedy_refresh: if True, expects dict output with keys {"prec","A_est","A_est_bin"}; handled specially
    """
    model: Any
    init: Dict[str, Any] = field(default_factory=dict)
    args: Dict[str, Any] = field(default_factory=dict)
    name: str = "baseline"
    standardize: bool = False
    adapt_lambda: bool = False
    topo_transpose: bool = False
    is_topogreedy_refresh: bool = False


# ---------------------------------------------
# Experiment runner
# ---------------------------------------------
@dataclass
class ExperimentConfig:
    n_graphs: int
    n_nodes: int
    n_samples_list: List[int]
    edge_threshold: float
    data_params: Dict[str, Any]
    baselines: List[BaselineSpec]
    out_dir: str = "./exp_results"
    run_tag: Optional[str] = None
    save_intermediate: bool = True
    seed_offset: int = 0  # to vary graph seeds across runs


class ExperimentRunner:
    def __init__(self, cfg: ExperimentConfig):
        self.cfg = cfg
        self.run_id = cfg.run_tag or dt.datetime.now().strftime("%Y%m%d_%H%M%S") + "_" + uuid.uuid4().hex[:6]
        self.out_root = Path(cfg.out_dir) / self.run_id
        self.out_root.mkdir(parents=True, exist_ok=True)
        self._save_manifest()

    # ---------- public API ----------
    def run(self, verbose: bool = True):
        """
        Orchestrates:
          - for each graph (with new simulation seed),
          - for each n_samples in list,
          - for each baseline,
            => train / estimate => metrics => save.

        Returns a dict of stacked arrays ready-to-plot.
        """
        B = len(self.cfg.baselines)
        S = len(self.cfg.n_samples_list)
        N = self.cfg.n_nodes
        G = self.cfg.n_graphs

        # Metrics tensors: (G, S, B)
        shd = np.zeros((G, S, B))
        tpr = np.zeros((G, S, B))
        fdr = np.zeros((G, S, B))
        fscore = np.zeros((G, S, B))
        err = np.zeros((G, S, B))
        runtime = np.zeros((G, S, B))
        dag_count = np.zeros((G, S, B))
        # Frobenius diff(Theta): normalized (G, S, B)
        theta_diff = np.zeros((G, S, B))

        # Store full adjacency estimates: (G, S, B, N, N)
        W_est_all = np.zeros((G, S, B, N, N))
        # Optional precision matrices when available (fill zeros otherwise)
        Theta_est_all = np.zeros((G, S, B, N, N))

        for g in range(G):
            if verbose:
                print(f"\n=== Graph {g+1}/{G} ===")

            # -------------------- simulate data for this graph --------------------
            graph_seed = self.cfg.seed_offset + g
            data_p = dict(self.cfg.data_params)
            data_p["n_nodes"] = self.cfg.n_nodes

            # we will re-simulate for each n_samples (fresh X but same W_true structure per spec)
            W_true_cache = None
            Theta_true_cache = None

            for si, n_samples in enumerate(self.cfg.n_samples_list):
                data_p_this = dict(data_p)
                data_p_this["n_samples"] = int(n_samples)

                # simulate SEM
                W_true, _, X, Theta_true = utils.simulate_sem(**data_p_this)

                # cache structure (if consistent across n_samples) — harmless if overwritten
                W_true_cache = W_true
                Theta_true_cache = Theta_true

                X_std = utils.standarize(X) if data_p_this.get("standarize", False) else X
                W_true_bin = utils.to_bin(W_true, self.cfg.edge_threshold)
                norm_W_true = np.linalg.norm(W_true)
                # emp_cov = (X_std.T @ X_std) / float(X_std.shape[0])
                emp_cov = np.cov(X_std, rowvar = False)
                print(f"cond: {np.linalg.cond(Theta_true)}")
                if verbose:
                    print(f"- samples={n_samples}, edges≈{np.count_nonzero(W_true_bin)}")

                # -------------------- run all baselines --------------------
                for bi, base in enumerate(self.cfg.baselines):
                    X_in = X_std if base.standardize else X
                    # args per baseline (copy so we can mutate lambdas safely)
                    args_call = dict(base.args)

                    # adaptive λ scheduling
                    if base.adapt_lambda:
                        if "lamb" in args_call:
                            args_call["lamb"] = get_lambda_value(self.cfg.n_nodes, n_samples, args_call["lamb"])
                        if "lambda1" in args_call:
                            args_call["lambda1"] = get_lambda_value(self.cfg.n_nodes, n_samples, args_call["lambda1"])

                    # ---------- execute baseline ----------
                    t0 = perf_counter()
                    W_est, Theta_est = self._run_one_baseline(
                        base=base,
                        X=X_in,
                        X_std=X_std,
                        emp_cov=emp_cov,
                        edge_thr=self.cfg.edge_threshold
                    )
                    t1 = perf_counter()
                    

                    # NaN safeguard
                    if np.isnan(W_est).any():
                        W_est = np.zeros_like(W_est)
                        W_bin = np.zeros_like(W_est)
                    else:
                        W_bin = utils.to_bin(W_est, self.cfg.edge_threshold)

                    # Optional orientation fix for TopoGreedy-like outputs
                    if base.topo_transpose:
                        W_est = W_est.T
                        W_bin = W_bin.T

                    # ---------- metrics ----------
                    shd[g, si, bi], tpr[g, si, bi], fdr[g, si, bi] = utils.count_accuracy(W_true_bin, W_bin)
                    fscore[g, si, bi] = f1_score(W_true_bin.flatten(), W_bin.flatten())
                    err[g, si, bi] = utils.compute_norm_sq_err(W_true, W_est, norm_W_true)
                    runtime[g, si, bi] = (t1 - t0)
                    dag_count[g, si, bi] = 1.0 if utils.is_dag(W_bin) else 0.0

                    # Theta Frobenius difference (if Theta_est available)
                    if Theta_est is None:
                        theta_diff[g, si, bi] = 0.0
                    else:
                        # num = np.linalg.norm(Theta_est - Theta_true, "fro")
                        # den = np.linalg.norm(Theta_true, "fro") + 1e-8
                        Theta_norm = np.linalg.norm(Theta_true, "fro")
                        theta_diff[g, si, bi] = utils.compute_norm_sq_err(Theta_true, Theta_est, Theta_norm)

                    # store estimates
                    W_est_all[g, si, bi] = W_est
                    if Theta_est is not None:
                        Theta_est_all[g, si, bi] = Theta_est

                    if verbose:
                        print(f"  · {base.name:<18s} | SHD {shd[g,si,bi]:.1f} | F1 {fscore[g,si,bi]:.3f} | "
                              f"ΘΔ {theta_diff[g,si,bi]:.3f} | {runtime[g,si,bi]:.2f}s")


                    # print(f"W_est: {W_est}")
                # optional per-(graph,samples) checkpoint save
                if self.cfg.save_intermediate:
                    data = self._save_block(
                        g=g, si=si,
                        W_true=W_true, Theta_true=Theta_true,
                        W_est_all=W_est_all[g, si],
                        Theta_est_all=Theta_est_all[g, si],
                        shd=shd[g, si], tpr=tpr[g, si], fdr=fdr[g, si],
                        f1=fscore[g, si], err=err[g, si], rt=runtime[g, si],
                        dags=dag_count[g, si], theta_diff=theta_diff[g, si]
                    )

        # ------------- stack & save final -------------
        final = dict(
            shd=shd, tpr=tpr, fdr=fdr, f1=fscore, err=err,
            runtime=runtime, dag_count=dag_count, theta_diff=theta_diff,
            W_est_all=W_est_all, Theta_est_all=Theta_est_all
        )
        np.savez_compressed(self.out_root / "final_results.npz", **final)
        if verbose:
            print(f"\nSaved results to: {self.out_root}")

        return final, data

    # ---------- internals ----------
    def _run_one_baseline(
        self,
        base: BaselineSpec,
        X: np.ndarray,
        X_std: np.ndarray,
        emp_cov: np.ndarray,
        edge_thr: float
    ) -> Tuple[np.ndarray, Optional[np.ndarray]]:
        """
        Runs a single baseline and returns (W_est, Theta_est or None).

        Special handling:
          - function baselines (e.g., notears_linear): W_est = f(X, **args)
          - TopoGreedy_refresh: expects dict with {'A_est','A_est_bin','prec'}
          - class baselines: model = cls(**init); model.fit(X, **args); W_est = model.W_est
            If model exposes Theta via .Theta_est (or .prec), we capture it.
        """
        # function-style baseline (no .fit)
        if callable(base.model) and not hasattr(base.model, "fit"):
            # handle TopoGreedy_refresh function signature:
            if base.is_topogreedy_refresh:
                out = base.model(X, emp_cov, **base.args)
                W_est = out.get("A_est", None)
                Theta_est = out.get("prec", None)
                if W_est is None:
                    W_est = np.zeros((X.shape[1], X.shape[1]))
                return W_est, Theta_est
            else:
                # generic function baseline like notears_linear(X, **args)
                W_est = base.model(X, **base.args)
                return W_est, None

        # class-style baseline
        model = base.model(**base.init) if base.init else base.model()
        # Try to pass standardized X only if the baseline asked for it (already handled by caller via X argument)
        model.fit(X, **base.args)

        # Extract adjacency
        W_est = getattr(model, "W_est", None)
        if W_est is None:
            # Some models might return directly from fit (rare); fallback to zeros
            W_est = np.zeros((X.shape[1], X.shape[1]))

        # Extract precision if present
        Theta_est = None
        if hasattr(model, "Theta_est"):
            Theta_est = getattr(model, "Theta_est")
        elif hasattr(model, "prec"):
            Theta_est = getattr(model, "prec")

        # TopoGreedy (non-refresh) variants sometimes return A transposed relative to convention;
        # caller can request topo_transpose=True in BaselineSpec to fix orientation.
        return W_est, Theta_est

    def _save_manifest(self):
        """Save config manifest (JSON) for reproducibility, without non-serializable fields."""
        # build a sanitized copy of the config:
        #   - convert baselines to lightweight dicts (stringify the model)
        sanitized_cfg = {
            "n_graphs": self.cfg.n_graphs,
            "n_nodes": self.cfg.n_nodes,
            "n_samples_list": list(self.cfg.n_samples_list),
            "edge_threshold": self.cfg.edge_threshold,
            "data_params": _to_jsonable(self.cfg.data_params),
            "baselines": [
                {
                    "name": b.name,
                    "model": _to_jsonable(b.model),           # "module.ClassOrFunc"
                    "init": _to_jsonable(b.init),
                    "args": _to_jsonable(b.args),
                    "standardize": b.standardize,
                    "adapt_lambda": b.adapt_lambda,
                    "topo_transpose": b.topo_transpose,
                    "is_topogreedy_refresh": b.is_topogreedy_refresh,
                }
                for b in self.cfg.baselines
            ],
            "out_dir": str(self.cfg.out_dir),
            "run_tag": self.cfg.run_tag,
            "save_intermediate": self.cfg.save_intermediate,
            "seed_offset": self.cfg.seed_offset,
        }

        manifest = {
            "run_id": self.run_id,
            "created_at": dt.datetime.now().isoformat(),
            "config": sanitized_cfg,
            "python": {"numpy_version": np.__version__},
        }

        (self.out_root / "config.json").write_text(json.dumps(_to_jsonable(manifest), indent=2))

    def _save_block(
        self, g: int, si: int,
        W_true: np.ndarray, Theta_true: np.ndarray,
        W_est_all: np.ndarray,
        Theta_est_all: np.ndarray,
        shd: np.ndarray, tpr: np.ndarray, fdr: np.ndarray,
        f1: np.ndarray, err: np.ndarray, rt: np.ndarray,
        dags: np.ndarray, theta_diff: np.ndarray
    ):
        """Persist per-(graph, sample-index) blob for debugging/inspection."""
        sub = self.out_root / f"graph_{g:03d}" / f"samples_{self.cfg.n_samples_list[si]}"
        sub.mkdir(parents=True, exist_ok=True)
        filename = sub / "block.npz"
        np.savez_compressed(
            filename,
            W_true=W_true, Theta_true=Theta_true,
            W_est_all=W_est_all, Theta_est_all=Theta_est_all,
            shd=shd, tpr=tpr, fdr=fdr, f1=f1, err=err, runtime=rt,
            dag_count=dags, theta_diff=theta_diff
        )
        
        # Return the complete data as a dictionary containing all arrays
        return {
            'filename': str(filename),
            'W_true': W_true,
            'Theta_true': Theta_true,
            'W_est_all': W_est_all,
            'Theta_est_all': Theta_est_all,
            'shd': shd,
            'tpr': tpr,
            'fdr': fdr,
            'f1': f1,
            'err': err,
            'runtime': rt,
            'dag_count': dags,
            'theta_diff': theta_diff
        }

